In [ ]:
import math

class CheckersGame:

    def __init__(self):
        self.board = [[' '] * 8 for _ in range(8)]
        self.current_player = 'black'  # starting player is black


        for row in range(3):
            for col in range(8):
                if row % 2 != col % 2:     #as pieces must be on black boxes
                    self.board[row][col] = 'b'  # 'b' represents black pieces, not king


        for row in range(5, 8):
            for col in range(8):
                if row % 2 != col % 2:   #as pieces must be on black boxes, there are spaces in between the pieces
                    self.board[row][col] = 'r'  # 'r' represents red pieces

    def is_valid_move(self, start, end, player):
        row_start, col_start, row_end, col_end = start[0], start[1], end[0], end[1]

        if not (0 <= row_end < 8 and 0 <= col_end < 8):
            return False

        if self.board[row_end][col_end] != ' ':
            return False

        # Determine direction based on player (1 for black moving down, -1 for red moving up)
        direction = 1 if player == 'black' else -1

        # Regular move (one step diagonal)
        if abs(col_end - col_start) == 1 and row_end == row_start + direction:
            return True

        # Capture move (jump over opponent's piece)
        if abs(col_end - col_start) == 2 and row_end == row_start + 2 * direction:
            mid_row = row_start + direction
            mid_col = (col_start + col_end) // 2

            if self.board[mid_row][mid_col] == ('r' if player == 'black' else 'b'):
                return True

        return False

    def make_move(self, start, end):
        row_start, col_start, row_end, col_end = start[0], start[1], end[0], end[1]
        self.board[row_end][col_end] = self.board[row_start][col_start]
        self.board[row_start][col_start] = ' '

        # Check for king promotion
        if (row_end == 0 and self.current_player == 'red') or (row_end == 7 and self.current_player == 'black'):
            self.board[row_end][col_end] = self.board[row_end][col_end].upper()  # Promote to king

        self.current_player = 'red' if self.current_player == 'black' else 'black'

    def get_valid_moves(self, player):
        moves = []
        for row in range(8):
            for col in range(8):
                if self.board[row][col].lower() == player[0]:  # Check if piece belongs to current player
                    directions = [1] if player == 'black' else [-1]
                    if self.board[row][col].isupper():
                        directions = [1, -1]  # King can move both forward and backward

                    for direction in directions:
                        for dcol in [-1, 1]:  # Check left and right diagonal moves
                            new_row, new_col = row + direction, col + dcol
                            if self.is_valid_move((row, col), (new_row, new_col), player):
                                moves.append((row, col, new_row, new_col))

        return moves

    def evaluate_board(self):
        score_black = sum(row.count('b') + row.count('B') for row in self.board)
        score_red = sum(row.count('r') + row.count('R') for row in self.board)
        return score_black - score_red

    def game_over(self):
        return not self.get_valid_moves('black') or not self.get_valid_moves('red')

    def alpha_beta_pruning(self, depth, alpha, beta, maximizing_player):
        if self.game_over() or depth == 0:
            return self.evaluate_board()

        if maximizing_player:
            max_eval = -math.inf
            for move in self.get_valid_moves(self.current_player):
                self.make_move((move[0], move[1]), (move[2], move[3]))
                eval = self.alpha_beta_pruning(depth - 1, alpha, beta, False)
                self.make_move((move[2], move[3]), (move[0], move[1]))
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = math.inf
            for move in self.get_valid_moves(self.current_player):
                self.make_move((move[0], move[1]), (move[2], move[3]))
                eval = self.alpha_beta_pruning(depth - 1, alpha, beta, True)
                self.make_move((move[2], move[3]), (move[0], move[1]))
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    def play(self):

        while not self.game_over():  #while the conditions
            valid_moves = self.get_valid_moves(self.current_player)  #get available moves for the current player on the board

            if not valid_moves:
                self.current_player = 'red' if self.current_player == 'black' else 'black'
                continue

            best_move = None
            max_eval = -math.inf
            alpha = -math.inf
            beta = math.inf

            for move in valid_moves:
                self.make_move((move[0], move[1]), (move[2], move[3]))
                eval = self.alpha_beta_pruning(3, alpha, beta, False)
                self.make_move((move[2], move[3]), (move[0], move[1]))

                if eval > max_eval:
                    max_eval = eval
                    best_move = move

                alpha = max(alpha, eval)

            if best_move:
                self.make_move((best_move[0], best_move[1]), (best_move[2], best_move[3]))
                self.print_board()

        winner = 'black' if self.evaluate_board() > 0 else 'red'
        print(f"Game over! {winner} wins.")

    def print_board(self):
        print("   0  1  2  3  4  5  6  7")
        for i, row in enumerate(self.board):
            print(f"{i} {' | '.join(row)}")   #row being dispalyed
        print()

def main():
    game = CheckersGame()
    game.print_board()
    game.play()

if __name__ == '__main__':
    main()


   0  1  2  3  4  5  6  7
0   | b |   | b |   | b |   | b
1 b |   | b |   | b |   | b |  
2   | b |   | b |   | b |   | b
3   |   |   |   |   |   |   |  
4   |   |   |   |   |   |   |  
5 r |   | r |   | r |   | r |  
6   | r |   | r |   | r |   | r
7 r |   | r |   | r |   | r |  

   0  1  2  3  4  5  6  7
0   | b |   | b |   | b |   | b
1 b |   | b |   | b |   | b |  
2   |   |   | b |   | b |   | b
3 b |   |   |   |   |   |   |  
4   |   |   |   |   |   |   |  
5 r |   | r |   | r |   | r |  
6   | r |   | r |   | r |   | r
7 r |   | r |   | r |   | r |  

   0  1  2  3  4  5  6  7
0   | B |   | B |   | b |   | b
1 b |   | b |   | b |   | b |  
2   |   |   | b |   | b |   | b
3 b |   |   |   |   |   |   |  
4   | r |   |   |   |   |   |  
5   |   | r |   | r |   | r |  
6   | r |   | r |   | r |   | r
7 r |   | r |   | r |   | r |  

   0  1  2  3  4  5  6  7
0   | B |   | B |   | b |   | b
1   |   | b |   | b |   | b |  
2   | b |   | b |   | b |   | b
3 b |   |   |   |   |   |   |